In [25]:
import requests
from bs4 import BeautifulSoup as bs 
import pandas as pd

In [26]:
#Output files
match_links = r'D:\My Projects\VS Code\Test codes\IPL2024_Match Links.xlsx'
match_information = r'D:\My Projects\VS Code\Test codes\IPL2024_Match_information.xlsx'
match_date_time = r'D:\My Projects\VS Code\Test codes\IPL2024_Match_Date&Time details.xlsx'
match_innings_details = r'D:\My Projects\VS Code\Test codes\IPL2024_Innings_Details.xlsx'


In [27]:
URL = 'https://www.cricbuzz.com/cricket-series/7607/indian-premier-league-2024/matches'

In [28]:
page = requests.get(URL)
page
# IF Response [200], proceed to next step

soup = bs(page.text, 'html')
#print(soup)
matches = soup.find_all('a', class_ ='text-hvr-underline' )
#matches

In [29]:
match_data = []
for match in matches:
    if 'title' in match.attrs:
        title = match['title']
        link = "https://www.cricbuzz.com" + match['href']

        base_url = "https://www.cricbuzz.com/live-cricket-scorecard"
        modified_link = match['href'].split("/cricket-scores")[1]
        scorecard_url = base_url + modified_link
            
        match_info = match.span.text
        teams = title.split(" Live Cricket Score")[0].split(" vs ")
        match_no = match_info.split(", ")[-1]
            
        match_data.append({
            "Match No": match_no,
            "Team1": teams[0],
            "Team2": teams[1],
            "Link": link,
            "Scorecard URL" : scorecard_url
            })
    
df = pd.DataFrame(match_data)
    
df.to_excel(match_links, index=False)
print("Successfully Scraped")

Successfully Scraped


Match Information

In [30]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url_df = pd.read_excel(match_links)

urls = url_df['Scorecard URL'].tolist()

excel_data = {
    "Match": [],
    "Date": [],
    "Toss": [],
    "Time": [],
    "Venue": [],
    "Umpires": [],
    "Third Umpire": [],
    "Match Referee": [],
    "Team": [],
    "Role": [],
    "Players": []
}

for url in urls:
    print(f"Processing URL: {url}")
    
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Failed to load page with status code: {response.status_code}")
        continue

    soup = BeautifulSoup(response.content, 'html.parser')

    match_info = {}

    # Match
    match = soup.find("div", class_="cb-col cb-col-73")
    match_info["Match"] = match.get_text(strip=True) if match else "N/A"

    # Date
    date_div = soup.find("div", text="Date")
    date = date_div.find_next_sibling("div").get_text(strip=True) if date_div else "N/A"
    match_info["Date"] = date

    # Toss
    toss_div = soup.find("div", text="Toss")
    toss = toss_div.find_next_sibling("div").get_text(strip=True) if toss_div else "N/A"
    match_info["Toss"] = toss

    # Time
    time_div = soup.find("div", text="Time")
    time = time_div.find_next_sibling("div").get_text(strip=True) if time_div else "N/A"
    match_info["Time"] = time

    # Venue
    venue_div = soup.find("div", text="Venue")
    venue = venue_div.find_next_sibling("div").get_text(strip=True) if venue_div else "N/A"
    match_info["Venue"] = venue

    # Umpires
    umpires_div = soup.find("div", text="Umpires")
    umpires = umpires_div.find_next_sibling("div").get_text(strip=True) if umpires_div else "N/A"
    match_info["Umpires"] = umpires

    # Third Umpire
    third_umpire_div = soup.find("div", text="Third Umpire")
    third_umpire = third_umpire_div.find_next_sibling("div").get_text(strip=True) if third_umpire_div else "N/A"
    match_info["Third Umpire"] = third_umpire

    # Match Referee
    referee_div = soup.find("div", text="Match Referee")
    referee = referee_div.find_next_sibling("div").get_text(strip=True) if referee_div else "N/A"
    match_info["Match Referee"] = referee

    # Player squads
    squads = {}
    team_sections = soup.find_all("div", class_="cb-minfo-tm-nm")
    
    for team_section in team_sections:
        team_name = team_section.get_text(strip=True).replace('Squad', '').strip()
        squads[team_name] = {}
        
        next_divs = team_section.find_next_siblings("div", class_="cb-minfo-tm-nm")
        
        if len(next_divs) >= 3:
            # Playing XI
            playing_div = next_divs[0].find("div", class_="cb-col cb-col-73")
            squads[team_name]["Playing"] = [a.get_text(strip=True) for a in playing_div.find_all("a")] if playing_div else []

            # Bench
            bench_div = next_divs[1].find("div", class_="cb-col cb-col-73")
            squads[team_name]["Bench"] = [a.get_text(strip=True) for a in bench_div.find_all("a")] if bench_div else []

            # Support Staff
            support_staff_div = next_divs[2].find("div", class_="cb-col cb-col-73")
            squads[team_name]["Support Staff"] = [a.get_text(strip=True) for a in support_staff_div.find_all("a")] if support_staff_div else []
        else:
            squads[team_name]["Playing"] = []
            squads[team_name]["Bench"] = []
            squads[team_name]["Support Staff"] = []

    match_info["Squads"] = squads

    for key, value in match_info.items():
        if key == "Squads":
            for team, players in value.items():
                for role, names in players.items():
                    excel_data["Match"].append(match_info["Match"])
                    excel_data["Date"].append(match_info["Date"])
                    excel_data["Toss"].append(match_info["Toss"])
                    excel_data["Time"].append(match_info["Time"])
                    excel_data["Venue"].append(match_info["Venue"])
                    excel_data["Umpires"].append(match_info["Umpires"])
                    excel_data["Third Umpire"].append(match_info["Third Umpire"])
                    excel_data["Match Referee"].append(match_info["Match Referee"])
                    excel_data["Team"].append(team)
                    excel_data["Role"].append(role)
                    excel_data["Players"].append(", ".join(names))

df = pd.DataFrame(excel_data)
df.to_excel(match_information, index=False)

print("Successfully Scraped")


Processing URL: https://www.cricbuzz.com/live-cricket-scorecard/89654/csk-vs-rcb-1st-match-indian-premier-league-2024


C:\Users\ASUS\AppData\Local\Temp\ipykernel_1192\2057705262.py:41: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  date_div = soup.find("div", text="Date")
C:\Users\ASUS\AppData\Local\Temp\ipykernel_1192\2057705262.py:46: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  toss_div = soup.find("div", text="Toss")
C:\Users\ASUS\AppData\Local\Temp\ipykernel_1192\2057705262.py:51: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  time_div = soup.find("div", text="Time")
C:\Users\ASUS\AppData\Local\Temp\ipykernel_1192\2057705262.py:56: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  venue_div = soup.find("div", text="Venue")
C:\Users\ASUS\AppData\Local\Temp\ipykernel_1192\2057705262.py:61: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' inste

Processing URL: https://www.cricbuzz.com/live-cricket-scorecard/89661/pbks-vs-dc-2nd-match-indian-premier-league-2024
Processing URL: https://www.cricbuzz.com/live-cricket-scorecard/89665/kkr-vs-srh-3rd-match-indian-premier-league-2024
Processing URL: https://www.cricbuzz.com/live-cricket-scorecard/89668/rr-vs-lsg-4th-match-indian-premier-league-2024
Processing URL: https://www.cricbuzz.com/live-cricket-scorecard/89675/gt-vs-mi-5th-match-indian-premier-league-2024
Processing URL: https://www.cricbuzz.com/live-cricket-scorecard/89682/rcb-vs-pbks-6th-match-indian-premier-league-2024
Processing URL: https://www.cricbuzz.com/live-cricket-scorecard/89689/csk-vs-gt-7th-match-indian-premier-league-2024
Processing URL: https://www.cricbuzz.com/live-cricket-scorecard/89693/srh-vs-mi-8th-match-indian-premier-league-2024
Processing URL: https://www.cricbuzz.com/live-cricket-scorecard/89700/rr-vs-dc-9th-match-indian-premier-league-2024
Processing URL: https://www.cricbuzz.com/live-cricket-scorecar

Match Details

In [31]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

file_path = match_links
df = pd.read_excel(file_path)

all_match_details = []

for index, row in df.iterrows():
    first_link = row['Link']
    
    try:
        page = requests.get(first_link)
        page.raise_for_status()
        soup = bs(page.text, 'html.parser') 

        match_header = soup.find('h1', class_='cb-nav-hdr cb-font-18 line-ht24')
        match = match_header.get_text().strip() if match_header else "Match Not Found"
        
        match_details = soup.find_all('div', class_='cb-nav-subhdr cb-font-12')
        
        match_texts = [element.get_text() for element in match_details]

        if match_texts:  
            match_text = match_texts[0]  

            # Extract Series, Venue, and Date & Time
            series = match_text.split("Series:")[1].split("Venue:")[0].strip()
            venue = match_text.split("Venue:")[1].split("Date & Time:")[0].strip()
            date_time = match_text.split("Date & Time:")[1].strip()

            all_match_details.append({
                "Match": match,
                "Series": series,
                "Venue": venue,
                "Date & Time": date_time
            })
        
        print(f"Processed link {index + 1}/{len(df)}: {first_link}")

    except Exception as e:
        print(f"Error processing link {first_link}: {e}")

final_df = pd.DataFrame(all_match_details)

final_df.to_excel(match_date_time, index=False)
print("Successfully Scraped")


Processed link 1/74: https://www.cricbuzz.com/cricket-scores/89654/csk-vs-rcb-1st-match-indian-premier-league-2024
Processed link 2/74: https://www.cricbuzz.com/cricket-scores/89661/pbks-vs-dc-2nd-match-indian-premier-league-2024
Processed link 3/74: https://www.cricbuzz.com/cricket-scores/89665/kkr-vs-srh-3rd-match-indian-premier-league-2024
Processed link 4/74: https://www.cricbuzz.com/cricket-scores/89668/rr-vs-lsg-4th-match-indian-premier-league-2024
Processed link 5/74: https://www.cricbuzz.com/cricket-scores/89675/gt-vs-mi-5th-match-indian-premier-league-2024
Processed link 6/74: https://www.cricbuzz.com/cricket-scores/89682/rcb-vs-pbks-6th-match-indian-premier-league-2024
Processed link 7/74: https://www.cricbuzz.com/cricket-scores/89689/csk-vs-gt-7th-match-indian-premier-league-2024
Processed link 8/74: https://www.cricbuzz.com/cricket-scores/89693/srh-vs-mi-8th-match-indian-premier-league-2024
Processed link 9/74: https://www.cricbuzz.com/cricket-scores/89700/rr-vs-dc-9th-matc

Scorecard Data Extraction

In [32]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


url_df = pd.read_excel(match_links)

urls = url_df['Scorecard URL'].tolist()

all_innings_data = []

for url in urls:
    print(f"Scraping data from: {url}")
    
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        header = soup.find('h1', class_='cb-nav-hdr cb-font-18 line-ht24').text.strip()

        innings_divs = soup.find_all('div', class_='cb-col cb-col-100 cb-ltst-wgt-hdr')
        
        for innings_div in innings_divs:
            data = innings_div.text.strip()
            all_innings_data.append((header, data))

    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

df = pd.DataFrame(all_innings_data, columns=['Match Title', 'Innings Details'])

output_file = match_innings_details
df.to_excel(output_file, index=False)

print("Successfully Scraped")


Scraping data from: https://www.cricbuzz.com/live-cricket-scorecard/89654/csk-vs-rcb-1st-match-indian-premier-league-2024
Scraping data from: https://www.cricbuzz.com/live-cricket-scorecard/89661/pbks-vs-dc-2nd-match-indian-premier-league-2024
Scraping data from: https://www.cricbuzz.com/live-cricket-scorecard/89665/kkr-vs-srh-3rd-match-indian-premier-league-2024
Scraping data from: https://www.cricbuzz.com/live-cricket-scorecard/89668/rr-vs-lsg-4th-match-indian-premier-league-2024
Scraping data from: https://www.cricbuzz.com/live-cricket-scorecard/89675/gt-vs-mi-5th-match-indian-premier-league-2024
Scraping data from: https://www.cricbuzz.com/live-cricket-scorecard/89682/rcb-vs-pbks-6th-match-indian-premier-league-2024
Scraping data from: https://www.cricbuzz.com/live-cricket-scorecard/89689/csk-vs-gt-7th-match-indian-premier-league-2024
Scraping data from: https://www.cricbuzz.com/live-cricket-scorecard/89693/srh-vs-mi-8th-match-indian-premier-league-2024
Scraping data from: https://